In [ ]:
import tensorflow as tf
print(tf.__version__)

import keras
from keras.layers import Input, Dense, Conv2D, Reshape, BatchNormalization, LeakyReLU, Conv2DTranspose
from keras.layers import Flatten, Reshape, LSTM
from keras.models import Model, Sequential
from keras.datasets import mnist
from keras.losses import binary_crossentropy
from keras import backend as K
from keras.optimizers import RMSprop, Adam

import numpy as np
import matplotlib.pyplot as plt 
import cv2
import skimage
import imutils
%matplotlib inline

np.random.seed(42)

from imutils import face_utils
import numpy as np
import argparse
import imutils
import dlib
import cv2

from skimage import img_as_float
from skimage.metrics import structural_similarity as ssim

In [ ]:
# load some intermediate and final results for evaluation

# load the cropped data
data = np.load("/content/drive/MyDrive/cropped_data_large_fixed.npy")

# load the model and pre-trained weights
loaded_decoder = keras.models.load_model("/content/drive/MyDrive/gray_decoder_L8_Fixedcropped")
loaded_decoder.load_weights("/content/drive/MyDrive/decoder_gray_model_L8_Fixedcropped_weights.h5")
# loaded_decoder = keras.models.load_model("gray_decoder.h5")

# load the latent space representation
data_time = np.load("/content/drive/MyDrive/latent_space_representation_gray_L8_Fixedcropped.npy")

# load the prediction in the extend interval
y_pred_extend = np.load("/content/drive/MyDrive/y_pred_extend_Fixed8-1000-01.npy")

In [ ]:
# reconstruction and prediction

latent_space = 8

regen = []

for i in range(9000):
    latent = y_pred_extend[i]
    latent = np.reshape(latent, (1, latent_space))
    reconstruction = loaded_decoder.predict(latent)
    reconstruction = np.reshape(reconstruction, (reconstruction.shape[1], reconstruction.shape[2]))
    reconstruction = (reconstruction * 255).astype("uint8")
    regen.append(reconstruction)

# save the result
#np.save("/content/drive/MyDrive/compare_regen_grayLarge.npy",regen)

In [ ]:
regen = np.load("/content/drive/MyDrive/compare_regen_grayLarge.npy")

<h1><b>Conduct SSIM evaluation on the whole face (280*275)</h1></b>

In [ ]:
# calculate the SSIM value for each frame
ssim_list = []
x = []

for i in range(data.shape[0]):
  ssim_list.append(ssim(img_as_float(data[i]), img_as_float(regen[i])))
  x.append(i)

In [ ]:
# plot and save the SSIM value for evaluating the performance of reconstruction and prediction
plt.figure(figsize=(24,8))
plt.plot(x[0:7000], ssim_list[0:7000], label = 'reconstruction', c = 'cornflowerblue')
plt.plot(x[7000:7428], ssim_list[7000:7428], label = 'prediction', c = 'mediumpurple')
plt.title('Image similarity between the ground truth and the regeneration/prediction', fontsize = 14)
plt.ylabel('SSIM value', fontsize = 14)
plt.xlabel('Frame number', fontsize = 14)
plt.legend(loc = 'lower left', fontsize = 14)
plt.show()
#plt.savefig("filename.png")

<h1><b>Conduct the SSIM evaluation on some parts of the face<\h1><\b>

In [ ]:
# plot the selected parts
# by using a software named Fiji, we obtained the coordinates for corresponding regions

plt.figure(figsize=(20,8))
img = data[7427]

plt.subplot(241)
x, y, w, h = 37, 91, 195, 49
plt.title('eyes', fontsize = 14)
plt.imshow(img[y:y+h, x:x+w], cmap = 'gray')

plt.subplot(242)
x, y, w, h = 80, 136, 114, 50
plt.title('nose', fontsize = 14)
plt.imshow(img[y:y+h, x:x+w], cmap = 'gray')

plt.subplot(243)
x, y, w, h= 71, 188, 124, 56
plt.title('mouth', fontsize = 14)
plt.imshow(img[y:y+h, x:x+w], cmap = 'gray')

plt.subplot(244)
x, y, w, h = 45, 137, 55, 66
plt.title('face', fontsize = 14)
plt.imshow(img[y:y+h, x:x+w], cmap = 'gray')

plt.subplot(245)
x, y, w, h = 0, 104, 37, 96
plt.title('left ear', fontsize = 14)
plt.imshow(img[y:y+h, x:x+w], cmap = 'gray')

plt.subplot(246)
x, y, w, h = 237, 106, 38, 96
plt.title('right ear', fontsize = 14)
plt.imshow(img[y:y+h, x:x+w], cmap = 'gray')

In [ ]:
def get_partial_SSIM (region):
    if region == 'eye':
        x, y, w, h = 37, 91, 195, 49
    if region == 'nose':
        x, y, w, h = 80, 136, 114, 50
    if region == 'mouth':
        x, y, w, h= 71, 188, 124, 56
    if region == 'face':
        x, y, w, h = 45, 137, 55, 66
    if region == 'left_ear':
        x, y, w, h = 0, 104, 37, 96
    if region == 'right_ear':
        x, y, w, h = 237, 106, 38, 96

    ssim_partial = []
  
    for i in range(7000, data.shape[0]):
        ground_truth = data[i][y:y+h, x:x+w]
        pred = regen[i][y:y+h, x:x+w]
        ssim_partial.append(ssim(img_as_float(ground_truth), img_as_float(pred)))
    return ssim_partial

In [ ]:
# x-axis
xx = []
for i in range(7000, data.shape[0]):
    xx.append(i)

In [ ]:
# plot the score of parts where the SSIM score is higher than the face as a whole
plt.figure(figsize=(25,8))
plt.plot(xx, ssim_list[7000: data.shape[0]], label = 'all', linewidth = 4, c = 'dimgray')
plt.plot(xx, get_partial_SSIM('eye'), label = 'eye')
plt.plot(xx, get_partial_SSIM('nose'), label = 'nose')
plt.plot(xx, get_partial_SSIM('mouth'), label = 'mouth')
plt.plot(xx, get_partial_SSIM('face'), label = 'face')
plt.title('Image similarity between the ground truth and the prediction (after the 7000 frame)', fontsize = 14)
plt.ylabel('SSIM value', fontsize = 14)
plt.xlabel('Frame number', fontsize = 14)
plt.legend(loc = 'lower left', fontsize = 14)
plt.show()


In [ ]:
# plot the score of parts where the SSIM score is lower than the face as a whole
plt.figure(figsize=(25,8))
plt.plot(xx, ssim_list[7000: data.shape[0]], label = 'all', linewidth = 4, c = 'dimgray')
plt.plot(xx, get_partial_SSIM('left_ear'), label = 'left ear', c = 'pink')
plt.plot(xx, get_partial_SSIM('right_ear'), label = 'right ear', c = 'c')
plt.title('Image similarity between the ground truth and the prediction (after the 7000 frame)', fontsize = 14)
plt.ylabel('SSIM value', fontsize = 14)
plt.xlabel('Frame number', fontsize = 14)
plt.legend(loc = 'lower left', fontsize = 14)
plt.show()
